# Hackathon #3

Written by Eleanor Quint

Topics: 
- Saving and loading TensorFlow models
- Running TensorFlow-based Python programs on Crane
- Overfitting, regularization, and early stopping

This is all setup in a IPython notebook so you can run any code you want to experiment with. Feel free to edit any cell, or add some to run your own code.

In [ ]:
# We'll start with our library imports...
from __future__ import print_function

import os  # to work with file paths

import tensorflow as tf         # to specify and run computation graphs
import numpy as np              # for numerical operations taking place outside of the TF graph
import matplotlib.pyplot as plt # to draw plots

mnist_dir = '/work/cse496dl/shared/hackathon/03/mnist/'

In [ ]:
# extract our dataset, MNIST
train_images = np.load(mnist_dir + 'mnist_train_images.npy')
train_labels = np.load(mnist_dir + 'mnist_train_labels.npy')
test_images = np.load(mnist_dir + 'mnist_test_images.npy')
test_labels = np.load(mnist_dir + 'mnist_test_labels.npy')

In [ ]:
# Clear the graph
tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, 784], name='data_placeholder')
# use a single name scope for the model
with tf.name_scope('linear_model') as scope:
    hidden = tf.layers.dense(x, 200, activation=tf.nn.relu, name='hidden_layer')
    output = tf.layers.dense(hidden, 10, name='output_layer')
    tf.identity(output, name='model_output')

global_step_tensor = tf.get_variable('global_step', trainable=False, shape=[], initializer=tf.zeros_initializer)
saver = tf.train.Saver()

All the preceding code is copied (with small modifications) from hackathon 2. We'll use it to illustrate saving and loading. Some notable modifications are the declaration of the 'global_step_tensor', the addition of the 'model_output' identity operation (which adds to the graph, even though we don't save the handle), and the addition of 'saver'.

### Saving and Loading TensorFlow Models

To save a model with initialized variables, we use the [save method](https://www.tensorflow.org/api_docs/python/tf/train/Saver#save) of an instance of [tf.train.Saver](https://www.tensorflow.org/api_docs/python/tf/train/Saver). Notice that this returns the checkpoint path prefix which may be passed directly to `Saver`'s load functions.

In [ ]:
save_directory = './hackathon3_logs'
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    img, class_vec = session.run([x, output], {x: np.expand_dims(train_images[42], axis=0)})
    print(class_vec)
    imgplot = plt.imshow(img.reshape((28,28)))
    
    # the next lines save the graph and variables in save_directory 
    # as "mnist_inference.ckpt.meta" and "mnist_inference.ckpt"
    path_prefix = saver.save(session, os.path.join(save_directory, "mnist_classification"), global_step=global_step_tensor)

Now, we'll clear the graph and try to run a datum through the model.

In [ ]:
# Clear the graph
tf.reset_default_graph()
session = tf.Session()
graph = session.graph
# the following line fails because the placeholder tensor isn't in the graph anymore
# session.run(output, {x: np.expand_dims(train_images[42], axis=0)})
print(path_prefix)

If the last line is uncommented, we get what is essentially an "operation not found" error. Now let's load the graph structure we saved before with [tf.train.import_meta_graph](https://www.tensorflow.org/api_docs/python/tf/train/import_meta_graph) and then use [Saver.restore](https://www.tensorflow.org/api_docs/python/tf/train/Saver#restore) to load and initialize the variable values. We can get handles to the in-graph `Tensor`s with [Graph.get_tensor_by_name](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name) and passing the name of the tensor (which is differentiated from the name of the operation by the ":0", which denotes the 0th tensor output of the op). We can then run the operations as normal.

In [ ]:
# loading the meta graph re-creates the graph structure in the current session, and restore initializes saved variables
saver = tf.train.import_meta_graph(path_prefix + '.meta')
saver.restore(session, path_prefix)

# get handles to graph Tensors, noticing the use of name scope in retrieving model_output
x = graph.get_tensor_by_name('data_placeholder:0')
output = graph.get_tensor_by_name('linear_model/model_output:0')
print(graph.get_operations())

img, class_vec = session.run([x, output], {x: np.expand_dims(train_images[42], axis=0)})
print(class_vec)
imgplot = plt.imshow(img.reshape((28,28)))

### Running TensorFlow-based Python programs on Crane

Because these IPython notebooks are run on the Crane login node, we should not attempt to run more than a trivially sized program. This means that we are not allowed to run more than a few training steps of a small model. Larger jobs, like fully optimizing a model, must be submitted to Slurm, the job scheduling manager for the Crane node.

We're now going to open a terminal from Jupyter to run the following commands:


```
cd $WORK
cp /work/cse496dl/shared/hackathon/03/run_py_496dl.sh $WORK
cp /work/cse496dl/shared/hackathon/03/basic.py $WORK
```

I've distributed a file called, "run_py_496dl.sh". It is most of what is needed to submit a Python program with TensorFlow installed and running on GPU. It expects a python file with a main function, and submits the job using `sbatch`:

`sbatch ./run_py_496dl.sh basic.py`

The way I have it written, it also passes through all arguments that follow the `.py`. Let's submit a job and then go over the details of the submit script. You can check on the status of your pending and running jobs with `squeue -u <USERNAME>`, substituting your Crane username, and you can cancel jobs with `scancel <JOBID>`, substituting the job id displayed by `squeue`. For more details, please visit the [HCC docs](https://hcc-docs.unl.edu/).

### Overfitting, regularization, and early stopping

If we have enough parameters in our model, and little enough data, after a long period of training we begin to experience overfitting. Empirically, this is when the loss value of the data visible to the model in training drops significantly below the loss value of the data set aside for testing. It implies that the model is looking for patterns specific to the training data that won't generalize to future, unseen data. This is a problem.

Solutions? Here are some first steps to think about:

1. Get more data
2. Reduce the number of parameters in the model
3. Regularize the weight/bias parameters of the model
4. Regularize using dropout
5. Early Stopping

Let's re-specify the network with regularization from [dropout](https://www.tensorflow.org/api_docs/python/tf/layers/dropout). Other common regularizers can be found in [tf.contrib.layers](https://www.tensorflow.org/api_docs/python/tf/contrib/layers) as well.

In [ ]:
tf.reset_default_graph()

KEEP_PROB = 0.7
x = tf.placeholder(tf.float32, [None, 784], name='input_placeholder')
with tf.name_scope('linear_model') as scope:
    # as strange as it sounds, using dropout on the input sometimes helps
    dropped_input = tf.layers.dropout(x, KEEP_PROB)
    hidden = tf.layers.dense(dropped_input,
                             400,
                             activation=tf.nn.relu,
                             name='hidden_layer')
    dropped_hidden = tf.layers.dropout(hidden, KEEP_PROB)
    output = tf.layers.dense(dropped_hidden,
                             10,
                             name='output_layer')

Or, alternatively, a network using [tf.contrib.layers.l2_regularizer](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/l2_regularizer).

In [ ]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, 784], name='input_placeholder')
with tf.name_scope('linear_model') as scope:
    hidden = tf.layers.dense(x,
                             400,
                             kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=0.01),
                             bias_regularizer=tf.contrib.layers.l2_regularizer(scale=0.01),
                             activation=tf.nn.relu,
                             name='hidden_layer')
    output = tf.layers.dense(hidden,
                             10,
                             kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=0.01),
                             bias_regularizer=tf.contrib.layers.l2_regularizer(scale=0.01),
                             name='output_layer')

When using any amount of numerical regularization, it is important to add the values to the final loss function that is used in `minimize`, otherwise the regularizers do nothing. Built-in regularizers are automatically added to a list that can be retrieved with [tf.get_collection](https://www.tensorflow.org/api_docs/python/tf/get_collection) which takes a [GraphKey](https://www.tensorflow.org/api_docs/python/tf/GraphKeys) and returns a list of tensors.

In [ ]:
# define classification loss WITH regularization loss
# In our case, it's L2, but could also commonly be L0, L1, or Linf
y = tf.placeholder(tf.float32, [None, 10], name='label')
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=output)

regularization_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
print(regularization_losses)
# this is the weight of the regularization part of the final loss
REG_COEFF = 0.001
# this value is what we'll pass to `minimize`
if regularization_losses:
    total_loss = cross_entropy + REG_COEFF * sum(regularization_losses)

Finally, the most important tool to combat overfitting is early stopping. This is the practice of saving copies of the parameters periodically and, after you've recognized that over-fitting is occurring (i.e., when the loss on the validation/test data doesn't decrease for a number of epochs), stop training and report the best saved copy of the parameters rather than the overfit version.

Whether deciding which set of parameters to use from training or trying to decide which form of regularization will work best by trying different kinds, you can get a better idea of how your regularization decisions will affect true generalization by further splitting the training data into training and validation sets. The validation data is left unused in adjusting model parameters, but is still used in training to make regularization decisions, and this leaves the test data to be the true measure of generalization.

You might use a function like the following for the task of splitting your data into two numpy arrays.

In [ ]:
def split_data(data, proportion):
    """
    Split a numpy array into two parts of `proportion` and `1 - proportion`
    
    Args:
        - data: numpy array, to be split along the first axis
        - proportion: a float less than 1
    """
    size = data.shape[0]
    split_idx = int(proportion * size)
    np.shuffle(data)
    return data[:split_idx], data[split_idx:]

## Hackathon 3 Exercise 1

Modify the `basic.py` file to add L2 regularization, split the training data to get a validation set, calculate loss on the validation (similar to test), and add early stopping. Train the model on Crane, submitting the job with `sbatch` and report the train, validation, and test loss values of the best set of parameters, along with the training epoch number they were saved from. (This is very similar to what you need to do for the first homework)

In [ ]:
### 1) MODIFY THE CODE, 2) TRAIN ON CRANE, 3) FILL THESE IN 4) SUBMIT THIS .IPYNB
# EPOCH: 
# TRAIN LOSS: 
# VALIDATION LOSS:
# TEST LOSS: